In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm_api
import statsmodels.formula.api as smf_api
import pandas as pd

from google.colab import files
uploaded = files.upload()

from warnings import filterwarnings
from sklearn import linear_model

Saving training.csv to training.csv


In [9]:
training_data = pd.read_csv("training.csv")
training_data.head()

,date,home_team,away_team,home_score,away_score,home_team_victory,draw,tournament_name,city,country,neutral,scoring_team,scoring_player,minute,own_goal,penalty,shootout_required,first_shooter,winner
0,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Knut Kroon,7.0,False,False,False,NaN,NaN
1,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Lennart Bunke,10.0,False,False,False,NaN,NaN
2,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Bertil Ericsson,13.0,False,False,False,NaN,NaN
3,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Sweden,Torsten Bunke,43.0,False,False,False,NaN,NaN
4,1933-06-11,Sweden,Estonia,6,2,True,False,FIFA World Cup qualification,Stockholm,Sweden,False,Estonia,Leonhard Kass,47.0,False,False,False,NaN,NaN


In [19]:
# prompt: Using dataframe training_data: linear regression

import statsmodels.formula.api as sm

# Define the linear regression model
# The model predicts 'home_score' based on 'away_score'
model = sm.ols('home_score ~ away_score', data=training_data)

# Fit the model to the data
results = model.fit()

# Print the regression results
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:             home_score   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     3596.
Date:                Sun, 09 Feb 2025   Prob (F-statistic):               0.00
Time:                        02:25:31   Log-Likelihood:                -55593.
No. Observations:               23719   AIC:                         1.112e+05
Df Residuals:                   23717   BIC:                         1.112e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.5096      0.021    164.916      0.0